# Topic Modelling of one million news headlines using LDA

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data.head(5)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [6]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
documents.head(5)
documents.shape

(1103665, 2)

### Data preprocessing.



In [ ]:
We will perform the following steps:

1. Tokenization : word counts are features. words are lowercased and punctuations are stripped off.
2. Remove words that have a length less than 3
3. Removal of stop words.
4. Words are lemmatized: words in third person are changed to first person
5. Words are stemmed : reduced to root form


In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /Users/gayatri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def lemmatize_and_stem(text):  ## lemmatize and stem the text.
    lemmatized_text = lemmatizer.lemmatize(text,'v')# lemmatized
    stemmer_text = stemmer.stem(lemmatized_text)  # stemmed
    return stemmer_text

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_and_stem(token))
    return result

In [25]:
doc_sample = documents[documents['index'] == 2010].values[0][0]
print('original document: ')
print(doc_sample)

words = []
for word in doc_sample.split(' '):
    words.append(word)
print("\nWords in the document")
print(words)

"""
Stop words are removed.
Words are lemmatized, tokenized and stemmed.
"""

print('\n\n Tokenized,Lemmatized and Stemmed document: ')
print(preprocess(doc_sample))

original document: 
gunners out to take advantage of absent enemies

Words in the document
['gunners', 'out', 'to', 'take', 'advantage', 'of', 'absent', 'enemies']


 Tokenized,Lemmatized and Stemmed document: 
['gunner', 'advantag', 'absent', 'enemi']


#####  Let us pre-process all the headlines.

In [26]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

# Creating a Bag of Words model from the processed news headlines

In [27]:
dictionary = gensim.corpora.Dictionary(processed_docs)   # dictionary will contain the number of times a word appears in the training set.


In [28]:
"""
We will filter out tokens whose frequency < 15
We will also filter out tokens that appear in more 0.5 times of the corpus size.
We will keep the top 100000
"""

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [39]:
"""
For each document we create a dictionary reporting how many words and how many times those words appear.
"""
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

[(76, 1), (112, 1), (483, 1), (4021, 1)]

In [47]:
bow_corpus[2010]

[(1722, 1), (2732, 1), (2733, 1), (2734, 1)]

# TF-IDF  

In [52]:
"""
Create tf-idf model object using models.

"""

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
print(tfidf)

TfidfModel(num_docs=1103665, num_nnz=4901459)


In [54]:
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903602896750699),
 (1, 0.38524510107363613),
 (2, 0.4974556071174764),
 (3, 0.5055678583740412)]


# Training LDA using Bag of words

In [55]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [56]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"queensland" + 0.021*"countri" + 0.019*"hospit" + 0.018*"tasmanian" + 0.017*"hour" + 0.015*"work" + 0.015*"children" + 0.013*"citi" + 0.011*"prison" + 0.010*"young"
Topic: 1 
Words: 0.030*"australia" + 0.023*"world" + 0.017*"market" + 0.012*"record" + 0.012*"share" + 0.011*"victoria" + 0.010*"industri" + 0.009*"melbourn" + 0.009*"port" + 0.009*"australian"
Topic: 2 
Words: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china" + 0.014*"west" + 0.014*"adelaid" + 0.014*"attack" + 0.013*"price" + 0.013*"gold"
Topic: 3 
Words: 0.027*"govern" + 0.017*"plan" + 0.014*"council" + 0.013*"rural" + 0.013*"say" + 0.012*"indigen" + 0.012*"turnbul" + 0.012*"water" + 0.011*"chang" + 0.011*"commun"
Topic: 4 
Words: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.014*"life" + 0.013*"lose" + 0.011*"john" + 0.009*"premier" + 0.009*"unit" + 0.008*"compani" + 0.008*"michael"
Topic: 5 
Words: 0.028*"elect" + 0.023*"south" + 0.020*"open" + 0.014*"australia" + 

# Testing LDA Usinf Bag of Words on Unseen document

In [57]:
processed_docs[4310]


['rain', 'help', 'dampen', 'bushfir']

In [58]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.41999998688697815	 
Topic: 0.023*"hous" + 0.019*"die" + 0.016*"home" + 0.014*"crash" + 0.014*"break" + 0.013*"take" + 0.011*"lead" + 0.010*"near" + 0.010*"perth" + 0.009*"star"

Score: 0.4199971556663513	 
Topic: 0.027*"govern" + 0.017*"plan" + 0.014*"council" + 0.013*"rural" + 0.013*"say" + 0.012*"indigen" + 0.012*"turnbul" + 0.012*"water" + 0.011*"chang" + 0.011*"commun"

Score: 0.020001811906695366	 
Topic: 0.017*"brisban" + 0.016*"famili" + 0.014*"child" + 0.013*"report" + 0.013*"jail" + 0.011*"abus" + 0.011*"sentenc" + 0.011*"show" + 0.010*"victim" + 0.009*"releas"

Score: 0.020001016557216644	 
Topic: 0.034*"polic" + 0.018*"interview" + 0.018*"health" + 0.018*"sydney" + 0.017*"miss" + 0.014*"investig" + 0.013*"concern" + 0.011*"search" + 0.011*"call" + 0.011*"servic"

Score: 0.0200000312179327	 
Topic: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china" + 0.014*"west" + 0.014*"adelaid" + 0.014*"attack" + 0.013*"price" + 0.013*"gold"

Score: 0.01

# Training LDA Using TF-IDF

In [59]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.017*"market" + 0.014*"news" + 0.012*"rural" + 0.009*"price" + 0.009*"share" + 0.008*"rise" + 0.008*"street" + 0.007*"busi" + 0.006*"rat" + 0.006*"sexual"
Topic: 1 Word: 0.021*"countri" + 0.019*"hour" + 0.008*"weather" + 0.006*"ash" + 0.005*"cancer" + 0.004*"search" + 0.004*"tasmania" + 0.004*"northern" + 0.004*"cattl" + 0.004*"music"
Topic: 2 Word: 0.009*"royal" + 0.008*"donald" + 0.008*"sport" + 0.008*"christma" + 0.007*"abbott" + 0.007*"commiss" + 0.006*"monday" + 0.006*"octob" + 0.006*"rain" + 0.006*"dairi"
Topic: 3 Word: 0.007*"climat" + 0.007*"korea" + 0.007*"malcolm" + 0.007*"care" + 0.007*"august" + 0.006*"andrew" + 0.006*"chang" + 0.005*"john" + 0.005*"nurs" + 0.005*"say"
Topic: 4 Word: 0.019*"trump" + 0.008*"peopl" + 0.007*"queensland" + 0.006*"friday" + 0.006*"asylum" + 0.006*"stori" + 0.005*"david" + 0.005*"seeker" + 0.005*"toni" + 0.004*"blog"
Topic: 5 Word: 0.011*"govern" + 0.009*"elect" + 0.007*"health" + 0.007*"turnbul" + 0.007*"fund" + 0.006*"budget" + 

In [61]:
unseen_document = 'Murder on 27th Street'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.36666667461395264	 Topic: 0.017*"brisban" + 0.016*"famili" + 0.014*"child" + 0.013*"report" + 0.013*"jail"
Score: 0.36666667461395264	 Topic: 0.031*"charg" + 0.030*"court" + 0.023*"murder" + 0.022*"polic" + 0.019*"face"
Score: 0.03333333507180214	 Topic: 0.032*"queensland" + 0.021*"countri" + 0.019*"hospit" + 0.018*"tasmanian" + 0.017*"hour"
Score: 0.03333333507180214	 Topic: 0.030*"australia" + 0.023*"world" + 0.017*"market" + 0.012*"record" + 0.012*"share"
Score: 0.03333333507180214	 Topic: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china"
Score: 0.03333333507180214	 Topic: 0.027*"govern" + 0.017*"plan" + 0.014*"council" + 0.013*"rural" + 0.013*"say"
Score: 0.03333333507180214	 Topic: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.014*"life" + 0.013*"lose"
Score: 0.03333333507180214	 Topic: 0.028*"elect" + 0.023*"south" + 0.020*"open" + 0.014*"australia" + 0.013*"final"
Score: 0.03333333507180214	 Topic: 0.023*"hous" + 0.019*"die" + 0.016*"hom